In [2]:
import numpy as np

In [3]:
'''Step 1 - Reading Images and Preparing samples'''

import csv

# Dictionaries to save all 10 folds of positive and negative samples
positive_samples = {}
negative_samples = {}

# Read txt file for negative and positive pairs in the dataset
with open('../data/pairs.txt', newline = '') as pairs:  
    
    pair_reader = csv.reader(pairs, delimiter='\t')
    sample_size = next(pair_reader)
    
    number_of_folds = int(sample_size[0])
    fold_size = int(sample_size[1])
    
    for fold in range(number_of_folds):
        
        positive_samples[fold] = []
        negative_samples[fold] = []
        
        # Create list of filenames for all positive pairs
        for i in range(fold_size):
            pair = next(pair_reader)
            positive_samples[fold].append([pair[0] + '/' + pair[0] + '_' + '0'*(4-len(pair[1])) + pair[1],
                                     pair[0] + '/' + pair[0] + '_' + '0'*(4-len(pair[2])) + pair[2]])
        
        # Create list of filenames for all negative pairs
        for i in range(fold_size):
            pair = next(pair_reader)
            negative_samples[fold].append([pair[0] + '/' + pair[0] + '_' + '0'*(4-len(pair[1])) + pair[1],
                                     pair[2] + '/' + pair[2] + '_' + '0'*(4-len(pair[3])) + pair[3]])

In [1]:
'''Pretrained model - load facenet model from keras'''

from keras.models import load_model
model = load_model("../model/facenet_keras.h5")

In [4]:
'''Function - Dlib library for finding embeddings from images'''

from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# function to get embeddings for an image
def extract_embeddings(filename, image_size, model):

    image = Image.open(filename)
    pixels = asarray(image)

    # detect bounding box around the face from image using mtncc
    detector = MTCNN()
    results = detector.detect_faces(pixels)

    # Cropping face out of image using the detected bounding box
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    face = pixels[y1:y1 + height, x1:x1 + width]

    # Converting image data to a format readable by our model(facenet model) to get embeddings
    img = Image.fromarray(face)
    img = img.resize(image_size)
    face_crop = asarray(img)
    face_crop = face_crop.astype('float32')
    mean, std = face_crop.mean(), face_crop.std()
    face_crop = (face_crop - mean) / std
    samples = np.expand_dims(face_crop, axis=0)

    # Get embeddings
    embedding = model.predict(samples)
    return embedding

In [4]:
''' Step 2 - Changing the images into embeddings and saving them respectively in positive_emb 
and negative_emb dictionary'''

total_samples = 20

# Get emeddings for desired number(upto 300) of positive pairs
# using only samples from fold 1 since getting embeddings for all folds took lot of time
positive_emb = []
for pairs in positive_samples[0][:total_samples]:
    print('resolving ' + str(pairs))
    emb1 = extract_embeddings('../data/lfw/' + pairs[0] + '.jpg', (160,160), model)
    emb2 = extract_embeddings('../data/lfw/' + pairs[1] + '.jpg', (160,160), model)

    positive_emb.append([emb1,emb2])

# Get emeddings for desired number(upto 300) of positive pairs
negative_emb = []
for pairs in negative_samples[0][:total_samples]:
    print('resolving ' + str(pairs))
    emb1 = extract_embeddings('../data/lfw/' + pairs[0] + '.jpg', (160,160), model)
    emb2 = extract_embeddings('../data/lfw/' + pairs[1] + '.jpg', (160,160), model)

    negative_emb.append([emb1,emb2])

In [16]:
'''Step 3 - Finding L2 distance between the embeddings'''

X_d = []
for emb in positive_emb:
    X_d.append(np.linalg.norm(emb[0] - emb[1]))
    
for emb in negative_emb:
    X_d.append(np.linalg.norm(emb[0] - emb[1]))
    
X_d = np.expand_dims(np.array(X_d),axis=1)

In [17]:
'''Step 4 - Preparing inputs for difference in embeddings based logistic regression model''' 

X = np.empty((total_samples*2,128))
for emb in positive_emb:
    np.append(X,emb[0] - emb[1])
    
for emb in negative_emb:
    np.append(X,emb[0] - emb[1])

In [18]:
'''Step 5 - Prepare target variable'''
y = np.append(np.ones((total_samples,1)),np.zeros((total_samples,1)))

In [2]:
'''Step 6 -  Model 1 - Create the Logistic Regression model which takes input as the distance 
between the embeddings of two images and outputs the similarity score(probablity) and 
prediction (binary yes/no) between the two images'''

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logreg_dist = LogisticRegression()
scores = cross_val_score(logreg_dist, X_d, y, cv=5, scoring = 'accuracy')
scores.mean()

In [3]:
'''Step 7 -  Model 2 - Create the Logistic Regression model which takes input as the 
difference between the each field of the embeddings of two images and outputs the similarity 
score(probablity) and prediction (binary yes/no) between the two images'''

logreg_emb = LogisticRegression()
scores = cross_val_score(logreg_emb, X, y, cv=5, scoring = 'accuracy')
scores.mean()